In [3]:
import pandas as pd
import datetime as dt

excel_file = "online_retail_II.xlsx"
df_2009_2010 = pd.read_excel(excel_file, sheet_name="Year 2009-2010")
df_2010_2011 = pd.read_excel(excel_file, sheet_name="Year 2010-2011")

df = pd.concat([df_2009_2010, df_2010_2011], ignore_index=True)

In [6]:
print(df.columns)

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')


In [8]:
#Toplam harcama değişkeni oluşturma
df["TotalPrice"] = df["Quantity"]*df["Price"]

#Eksik değerlerin kontrolü ve temizliği
df.dropna(subset=["Customer ID"], inplace=True)

df.dropna(subset=["Invoice"], inplace=True) #recency hesaplaması için kritik.

print(df.isnull().sum())

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
TotalPrice     0
dtype: int64


In [9]:
df = df[~df["Invoice"].str.startswith("C", na=False)] #iade işlemlerini çıkardık. 

reference_date = df["InvoiceDate"].max() + pd.Timedelta(days=1) #recency hesaplaması için referans tarih. 

In [11]:
rfm = df.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (reference_date - x.max()).days,
    "Invoice": "nunique",
    "TotalPrice": "sum"
})

rfm.columns = ["Recency", "Frequency", "Monetary"]
rfm["Recency"] = rfm["Recency"].astype(int)

In [12]:
rfm["R_Score"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1])
rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["M_Score"] = pd.qcut(rfm["Monetary"], 5, labels=[1,2,3, 4,5])

In [16]:
rfm["RFM_Score"] = rfm["R_Score"].astype(str) + rfm["F_Score"].astype(str) + rfm["M_Score"].astype(str)

seg_map = {
    r'[1-2][1-2]': 'Hibernating',        # Düşük F, Düşük R -> Uyuyanlar
    r'[1-2][3-4]': 'At Risk',           # Yüksek F, Düşük R -> Risk Altında (Uzun süredir gelmemiş ama önceden sık geliyordu)
    r'[1-2]5': 'Cannot Lose Them',     # En Yüksek F, Düşük R -> Kaybedilmemesi Gerekenler
    r'3[1-2]': 'About To Sleep',       # Orta R, Düşük F -> Uyumak Üzere
    r'33': 'Need Attention',          # Orta R, Orta F -> İlgi Gerekenler
    r'[3-4][4-5]': 'Loyal Customers',     # Orta/Yüksek R, Yüksek F -> Sadık Müşteriler
    r'41': 'Promising',               # Yüksek R (Yeni), Düşük F -> Gelecek Vaat Edenler
    r'51': 'New Customers',           # En Yüksek R (Çok Yeni), Düşük F -> Yeni Müşteriler
    r'[4-5][2-3]': 'Potential Loyalists', # Yüksek R, Orta F -> Potansiyel Sadıklar
    r'[4-5][4-5]': 'Champions'            # En Yüksek R, En Yüksek F -> Şampiyonlar (En iyiler)
}
rfm["Segment_RF"] = rfm["R_Score"].astype(str) + rfm["F_Score"].astype(str)
rfm["Segment"] = rfm["Segment_RF"].replace(seg_map, regex=True)

In [17]:
print("\nMüşteri Segmentlerinin Dağılımı:")
print(rfm["Segment"].value_counts())


Müşteri Segmentlerinin Dağılımı:
Segment
Hibernating            1525
Loyal Customers        1161
Champions               837
At Risk                 753
Potential Loyalists     714
About To Sleep          384
Need Attention          267
Promising               114
Cannot Lose Them         71
New Customers            55
Name: count, dtype: int64


In [18]:
rfm.head()

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment,Segment_RF
Customer ID,,,,,,,,,
12346.0,326,12,77556.46,2,5,5,255,Cannot Lose Them,25
12347.0,2,8,5633.32,5,4,5,545,Champions,54
12348.0,75,5,2019.40,3,4,4,344,Loyal Customers,34
12349.0,19,4,4428.69,5,3,5,535,Potential Loyalists,53
12350.0,310,1,334.40,2,1,2,212,Hibernating,21


In [19]:
rfm_output_file = "rfm_analysis_results.csv"
rfm.reset_index().to_csv(rfm_output_file, index=False)
print(f"\nRFM analizi sonuçları {rfm_output_file} dosyasına kaydedildi")


RFM analizi sonuçları rfm_analysis_results.csv dosyasına kaydedildi
